In [21]:
import requests
import json
import sqlite3

# Note:  Would be good to capture date as-well and use it for querying purposes

# :Method: Send a request to The Odds API for the chosen sport's games.  Add each game's data (books/teams/prices) to an empty SQL table
# :params: key - API key, sport - Desired sport
def PullOdds(key, sport):
    # - Pull the chosen sport's odds for [today's] games
    
    url = f'https://api.the-odds-api.com/v4/sports/{sport}/odds/?regions=us&oddsFormat=american&apiKey={key}'

    response = requests.get(url)

    # - Check to make sure that we connected to the right endpoint
    if response.ok:
        pass
    else:
        return print("Invalid request")
    
    odds_data = json.loads(response.text)

    # - Connect to our database
    conn = sqlite3.connect('oddsDatabase.db')
    cur = conn.cursor()

    # - Loop through the data and add each matchup to our database
    drop_table_query = 'DROP TABLE IF EXISTS matchups'
    cur.execute(drop_table_query)

    # matchup: away@home
    create_table_query = '''
    CREATE TABLE matchups (
    matchup TEXT,
    away TEXT,
    awayPrice INTEGER,
    home TEXT,
    homePrice INTEGER,
    book TEXT
    )
    '''
    cur.execute(create_table_query)

    # Make it so that we drop table, create table, executemany for each game; and query the results for each game - Then bucket before moving to the next game

    matches = []
    for game in odds_data:
        matchupID = game['away_team'] + '@' + game['home_team']
        for matchups in game['bookmakers']:
            # These are strictly moneyline odds as of right now
            #print(matchups['title'], ' | ', 'Away:', matchups['markets'][0]['outcomes'][0]['name'], matchups['markets'][0]['outcomes'][0]['price'], ' | ', 'Home:', matchups['markets'][0]['outcomes'][1]['name'], matchups['markets'][0]['outcomes'][1]['price'])

            awayTeam = matchups['markets'][0]['outcomes'][0]['name']
            awayTeam = awayTeam.replace(" ", "_")
            awayPrice = matchups['markets'][0]['outcomes'][0]['price']
            
            homeTeam = matchups['markets'][0]['outcomes'][1]['name']
            homeTeam = homeTeam.replace(" ", "_")
            homePrice = matchups['markets'][0]['outcomes'][1]['price']
            
            book = matchups['title']

            matches.append( (matchupID, awayTeam, awayPrice, homeTeam, homePrice, book) )

        insert_match_query = '''INSERT INTO matchups (matchup, away, awayPrice, home, homePrice, book) VALUES (?, ?, ?, ?, ?, ?);'''
        
    cur.executemany(insert_match_query, matches)

    conn.commit()

# :Method: Query each game to display the top 3 books for each team for each game
def QueryTopThree():
    
    # - Connect to our database
    try:
        conn = sqlite3.connect('oddsDatabase.db')
        cur = conn.cursor()
    except:
        print("Could not connect to the database")
    
    # - Capture each unique matchup that we pulled
    select_matchups_query = '''
    SELECT DISTINCT matchup FROM matchups
    '''
    cur.execute(select_matchups_query)
    game_list = cur.fetchall()
    
    # - Access each game by their IDs and return the three best prices
    queried_awayPrices = []
    queried_homePrices = []
    for gameID in game_list:
        away_game_prices = []
        home_game_prices = []
        
        # - Select our best home prices
        select_best_homePrices_query = f'''
        SELECT book, home, homePrice from matchups  WHERE matchup = "{gameID[0]}"
        ORDER BY
        CASE
            WHEN (SELECT AVG(homePrice) FROM matchups) >= 0 THEN homePrice
            ELSE -homePrice
        END ASC
        LIMIT 3;
        '''
        cur.execute(select_best_homePrices_query)
        best_home_prices = cur.fetchall()

        # - Add our three picks to a list
        for row in best_home_prices:
            home_game_prices.append(row[0] + ": " + row[1] + " @ " + str(row[2]))
        
        # - Add our list of three picks, to our complete list of [today's] picks
        queried_homePrices.append(home_game_prices)

        # - Select our best away prices
        select_best_awayPrices_query = f'''
        SELECT book, away, awayPrice from matchups  WHERE matchup = "{gameID[0]}"
        ORDER BY
        CASE
            WHEN (SELECT AVG(awayPrice) FROM matchups) >= 0 THEN awayPrice
            ELSE -awayPrice
        END ASC
        LIMIT 3;
        '''
        cur.execute(select_best_awayPrices_query)
        best_away_prices = cur.fetchall()

        # - Add our three picks to a list
        for row in best_away_prices:
            away_game_prices.append(row[0] + ": " + row[1] + " @ " + str(row[2]))
        
        # - Add our list of three picks, to our complete list of [today's] picks
        queried_awayPrices.append(away_game_prices)
    
    print("================================")
    print("Away Plays")
    for item in queried_awayPrices:
        print(item)
    print("================================")
    print("Home Plays")
    for item in queried_homePrices:
        print(item)
        
# :Method: Query each game and display three buckets of value plays indicating the amount of value with each play (if any)
def QueryMispricedPlays():
    # for game in today's games:
    select_best_price_query = '''
    SELECT awayPrice FROM matchups ORDER BY awayPrice ASC -probably need to make it adjust like above... - LIMIT 1;
    '''

    select_second_best_price_query = '''
    SELECT awayPrice FROM matchups ORDER BY awayPrice ASC -probably need to make it adjust like above... - LIMIT 1 OFFSET 1;
    '''

    # fetchone

    pass

# key = 'fa8219191b40b3fec426e43e1a7a37b8'
# sport = 'baseball_mlb'

#PullOdds(key = 'fa8219191b40b3fec426e43e1a7a37b8', sport = 'baseball_mlb')
QueryTopThree()

Away Plays
['TwinSpires: Baltimore_Orioles @ 1300', 'Unibet: Baltimore_Orioles @ 1300', 'Barstool Sportsbook: Baltimore_Orioles @ 1300']
['BetMGM: Miami_Marlins @ 210', 'Bovada: Miami_Marlins @ 200', 'TwinSpires: Miami_Marlins @ 200']
['DraftKings: Cincinnati_Reds @ -160', 'Unibet: Cincinnati_Reds @ -165', 'Barstool Sportsbook: Cincinnati_Reds @ -165']
['Circa Sports: Chicago_Cubs @ -1429', 'FOX Bet: Chicago_Cubs @ -1667', 'DraftKings: Chicago_Cubs @ -1700']
['FOX Bet: Detroit_Tigers @ -333', 'Circa Sports: Detroit_Tigers @ -400', 'Barstool Sportsbook: Detroit_Tigers @ -435']
['BetRivers: Arizona_Diamondbacks @ -143', 'Barstool Sportsbook: Arizona_Diamondbacks @ -143', 'Unibet: Arizona_Diamondbacks @ -143']
['BetUS: Houston_Astros @ -134', 'LowVig.ag: Houston_Astros @ -135', 'BetOnline.ag: Houston_Astros @ -135']
['BetUS: St._Louis_Cardinals @ -150', 'LowVig.ag: St._Louis_Cardinals @ -152', 'BetOnline.ag: St._Louis_Cardinals @ -152']
['Circa Sports: Atlanta_Braves @ 122', 'LowVig.ag: A